In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.6'

# Keras Functional API

Let's first compare keras functional API with our Sequential model.

In [2]:
from keras import Input, layers
from keras.models import Sequential, Model

# Sequential
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Using functional API
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

model = Model(input_tensor, output_tensor)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

model.fit(x_train, y_train, epochs=10, batch_size=128)

score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 0s 144us/step - loss: 11.5766
Epoch 2/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5560
Epoch 3/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5491
Epoch 4/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5453
Epoch 5/10
1000/1000 [==============================] - 0s 17us/step - loss: 11.5419
Epoch 6/10
1000/1000 [==============================] - 0s 18us/step - loss: 11.5400
Epoch 7/10
1000/1000 [==============================] - 0s 21us/step - loss: 11.5380
Epoch 8/10
1000/1000 [==============================] - 0s 16us/step - loss: 11.5363
Epoch 9/10
1000/1000 [==============================] - 0s 16us/step - loss: 11.5351
Epoch 10/10
1000/1000 [==============================] - 0s 45us/step


# Multi-input models

The functional 
API
 can be used to build models that have multiple inputs. Typically,
such models at some point me
rge their different input bran
ches using a layer that can
combine several tensors: by 
adding them, concatenating them, and so on. This is usu-
ally  done  via  a  Keras  me
rge  operation  such  as  `keras.layers.add`
, `keras.layers
.concatenate`
, and so on. Let’s look at a very si
mple example of a multi-input model:
a question-answering model.

In [0]:
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [26]:
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

model.fit([text, question], answers, epochs=10, batch_size=128)

Epoch 1/10


InvalidArgumentError: ignored